In [ ]:
# 使用label变成-1，0，1三分类任务

In [1]:
import pandas as pd
from dateutil import parser
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
from tqdm import tqdm
import numpy as np
import torch.nn.functional as F

# Ensure CUDA_LAUNCH_BLOCKING is set for better debugging
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Define file paths
data_dir = "/data1/dxw_data/llm/redbook_final/script_next/"
rawdata_path = os.path.join(data_dir, "rawdata_20%.csv")
after2monthdata_path = os.path.join(data_dir, "after2monthdata_20%_with_trend.csv")
image_dir = os.path.join(data_dir, "data_img_20%")

# Read CSV files
rawdata = pd.read_csv(rawdata_path)
after2monthdata = pd.read_csv(after2monthdata_path)

# Convert date columns to standard format
def parse_date(date_str):
    try:
        return parser.parse(date_str)
    except ValueError:
        return None

rawdata['post_date'] = rawdata['post_date'].apply(parse_date)
rawdata = rawdata.dropna(subset=['post_date'])

# Split data into training and testing sets
train_rawdata = rawdata[(rawdata['post_date'].dt.month >= 1) & (rawdata['post_date'].dt.month <= 9)]
test_rawdata = rawdata[(rawdata['post_date'].dt.month >= 10) & (rawdata['post_date'].dt.month <= 12)]

train_after2monthdata = after2monthdata[after2monthdata['post_id'].isin(train_rawdata['post_id'])]
test_after2monthdata = after2monthdata[after2monthdata['post_id'].isin(test_rawdata['post_id'])]

# Remove non-finite values in the 'trend' column
train_after2monthdata = train_after2monthdata.replace([np.inf, -np.inf], np.nan).dropna(subset=['trend'])
test_after2monthdata = test_after2monthdata.replace([np.inf, -np.inf], np.nan).dropna(subset=['trend'])

# Ensure that the 'trend' column has correct integer labels
train_after2monthdata['trend'] = train_after2monthdata['trend'].astype(int)
test_after2monthdata['trend'] = test_after2monthdata['trend'].astype(int)

# Only use 1/10th of the data
def get_subset_indices(data, fraction=0.01):
    data_size = len(data)
    indices = list(range(data_size))
    np.random.shuffle(indices)
    split = int(np.floor(fraction * data_size))
    return indices[:split]

train_indices = get_subset_indices(train_rawdata)
test_indices = get_subset_indices(test_rawdata)

train_rawdata = train_rawdata.iloc[train_indices]
test_rawdata = test_rawdata.iloc[test_indices]

train_after2monthdata = train_after2monthdata[train_after2monthdata['post_id'].isin(train_rawdata['post_id'])]
test_after2monthdata = test_after2monthdata[test_after2monthdata['post_id'].isin(test_rawdata['post_id'])]

# Custom Dataset Class
class MultimodalDataset(Dataset):
    def __init__(self, rawdata, after2monthdata, image_dir, transform=None, max_images=1):
        self.rawdata = rawdata
        self.after2monthdata = after2monthdata
        self.image_dir = image_dir
        self.transform = transform
        self.max_images = max_images
        self.data = self._prepare_data()
    
    def _prepare_data(self):
        data = []
        for _, row in tqdm(self.rawdata.iterrows(), total=self.rawdata.shape[0], desc="Processing data"):
            poster_id = row['poster_id']
            post_id = row['post_id']
            image_files = [f for f in os.listdir(self.image_dir) if f"{poster_id}_{post_id}" in f]
            images = []
            for image_file in image_files[:self.max_images]:
                image_path = os.path.join(self.image_dir, image_file)
                image = Image.open(image_path).convert('RGB')
                if self.transform:
                    image = self.transform(image)
                images.append(image)
            # If less than max_images, pad with zeros
            while len(images) < self.max_images:
                images.append(torch.zeros((3, 224, 224)))
            if images:
                summary = row['summary']
                label_data = self.after2monthdata[self.after2monthdata['post_id'] == post_id]['trend']
                if not label_data.empty:
                    label = label_data.values[0]
                    # Convert label to one-hot encoding
                    one_hot_label = F.one_hot(torch.tensor(label + 1), num_classes=3)  # -1, 0, 1 -> 0, 1, 2
                    data.append((summary, images, one_hot_label))
        return data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        summary, images, label = self.data[idx]
        return summary, torch.stack(images).float(), label.float()

# Image Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Create Datasets
train_dataset = MultimodalDataset(train_rawdata, train_after2monthdata, image_dir, transform=transform)
test_dataset = MultimodalDataset(test_rawdata, test_after2monthdata, image_dir, transform=transform)

# Create Data Loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

Processing data: 100%|██████████| 80/80 [00:04<00:00, 17.02it/s]


In [2]:
# Model Definition
import torch
import torch.nn as nn
import torch.optim as optim
from imagebind import data
from imagebind.models import imagebind_model
from imagebind.models.imagebind_model import ModalityType
from tqdm import tqdm

device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Load imagebind model
imagebind_model = imagebind_model.imagebind_huge(pretrained=True)
imagebind_model.eval()
imagebind_model.to(device)

class CrossAttentionFusion(nn.Module):
    def __init__(self, text_embedding_dim, vision_embedding_dim, common_embedding_dim, num_heads):
        super(CrossAttentionFusion, self).__init__()
        self.text_linear = nn.Linear(text_embedding_dim, common_embedding_dim)
        self.vision_linear = nn.Linear(vision_embedding_dim, common_embedding_dim)
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=common_embedding_dim, nhead=num_heads), num_layers=2)
        self.lstm = nn.LSTM(common_embedding_dim, common_embedding_dim, batch_first=True)
        self.fc = nn.Linear(common_embedding_dim, 3)  # Three classes

    def forward(self, text_embeddings, vision_embeddings):
        text_embeddings = self.text_linear(text_embeddings)
        vision_embeddings = self.vision_linear(vision_embeddings)
        # Flatten embeddings to match Transformer input requirements
        text_embeddings = text_embeddings.view(text_embeddings.size(0), -1, text_embeddings.size(-1))
        vision_embeddings = vision_embeddings.view(vision_embeddings.size(0), -1, vision_embeddings.size(-1))
        multimodal_embeddings = torch.cat((text_embeddings, vision_embeddings), dim=1)
        multimodal_embeddings = self.transformer_encoder(multimodal_embeddings)
        lstm_out, _ = self.lstm(multimodal_embeddings)
        lstm_out = lstm_out[:, -1, :]  # Take output of the last time step
        output = self.fc(lstm_out)
        return output

def get_embeddings(text_list, image_tensors):
    inputs = {
        ModalityType.TEXT: data.load_and_transform_text(text_list, device),
        ModalityType.VISION: image_tensors.to(device)
    }
    
    with torch.no_grad():
        embeddings = imagebind_model(inputs)
    
    # Check for NaNs and Infinities in embeddings
    if torch.isnan(embeddings[ModalityType.TEXT]).any() or torch.isinf(embeddings[ModalityType.TEXT]).any():
        print("NaN or Infinity values found in text embeddings")
    if torch.isnan(embeddings[ModalityType.VISION]).any() or torch.isinf(embeddings[ModalityType.VISION]).any():
        print("NaN or Infinity values found in vision embeddings")
    
    return embeddings

def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    log_file = open("log.txt", "w")
    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
            summaries, images, labels = batch
            images, labels = images.to(device), labels.to(device)
            text_embeddings = get_embeddings(summaries, images)
            optimizer.zero_grad()
            outputs = model(text_embeddings[ModalityType.TEXT], text_embeddings[ModalityType.VISION])
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        epoch_loss = running_loss / len(train_loader)
        log_file.write(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss}\n')
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss}')
    log_file.close()

# Initialize model, criterion, and optimizer
text_embedding_dim = 1024
vision_embedding_dim = 1024
common_embedding_dim = 768
num_heads = 8

model = CrossAttentionFusion(text_embedding_dim, vision_embedding_dim, common_embedding_dim, num_heads).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
train_model(model, train_loader, criterion, optimizer, num_epochs)

# Save the model
torch.save(model.state_dict(), os.path.join(data_dir, "multimodal_model.pth"))

print("Model training completed and saved!")

/home/dxw/anaconda3/envs/agent/lib/python3.8/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
/home/dxw/anaconda3/envs/agent/lib/python3.8/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/home/dxw/anaconda3/envs/agent/lib/python3.8/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warning

Epoch [1/10], Loss: 0.6163473397493362


Training Epoch 2/10: 100%|██████████| 10/10 [00:05<00:00,  1.99it/s]


Epoch [2/10], Loss: 0.5370518237352371


Training Epoch 3/10: 100%|██████████| 10/10 [00:04<00:00,  2.01it/s]


...

Epoch [8/10], Loss: 0.43335051089525223


Training Epoch 9/10: 100%|██████████| 10/10 [00:04<00:00,  2.02it/s]


Epoch [9/10], Loss: 0.4291202686727047


Training Epoch 10/10: 100%|██████████| 10/10 [00:05<00:00,  1.99it/s]


Epoch [10/10], Loss: 0.5224921770393849
Model training completed and saved!


In [3]:
import torch
import torch.nn.functional as F
from sklearn.metrics import accuracy_score

# Define the function to evaluate the model
def evaluate_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluating"):
            summaries, images, labels = batch
            images, labels = images.to(device), labels.to(device)
            text_embeddings = get_embeddings(summaries, images)
            outputs = model(text_embeddings[ModalityType.TEXT], text_embeddings[ModalityType.VISION])
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(torch.argmax(labels, dim=1).cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    print(f'Accuracy: {accuracy:.4f}')
    return accuracy

# Load the trained model
model_path = os.path.join(data_dir, "multimodal_model.pth")
model = CrossAttentionFusion(text_embedding_dim, vision_embedding_dim, common_embedding_dim, num_heads).to(device)
model.load_state_dict(torch.load(model_path))

# Evaluate the model on the test set
test_accuracy = evaluate_model(model, test_loader)


Evaluating: 100%|██████████| 3/3 [00:01<00:00,  1.97it/s]

Accuracy: 0.9756
